In [1]:
import os
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
import torchvision.transforms as T
import albumentations as A
from albumentations.pytorch import ToTensorV2
import json

/home/the/miniconda/envs/ml2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ANN_DIR = "./ds/ann"
IMG_DIR = "./ds/img"

In [5]:
records = []

for fname in os.listdir(ANN_DIR):
    if fname.endswith(".json"):
        fpath = os.path.join(ANN_DIR, fname)
        with open(fpath, "r") as f:
            data = json.load(f)
        
        image_id = fname.replace(".jpg.json", "")
        
        # Extract all class titles from objects
        objects = data.get("objects", [])
        class_titles = [obj.get("classTitle") for obj in objects if obj.get("classTitle")]
        
        if class_titles:
            # Use the first class title or join multiple
            dx = class_titles[0]  # or ", ".join(class_titles) for multiple
            records.append({"image_id": image_id, "dx": dx})

In [6]:
# Convert to DataFrame
df = pd.DataFrame(records)
print(df.head())
print(df["dx"].value_counts())

       image_id                dx
0  ISIC_0025149  melanocytic nevi
1  ISIC_0026719  melanocytic nevi
2  ISIC_0028157  melanocytic nevi
3  ISIC_0028285          melanoma
4  ISIC_0027353  melanocytic nevi
dx
melanocytic nevi                 6705
melanoma                         1112
benign keratosis-like lesions    1098
basal cell carcinoma              514
actinic keratoses                 327
vascular lesions                  142
dermatofibroma                    115
Name: count, dtype: int64


In [7]:
# Save for later
df.to_csv("HAM10000_metadata.csv", index=False)

In [8]:
# Paths
IMG_DIR = "./ds/img"
ANN_PATH = "./HAM10000_metadata.csv"

In [9]:
# Load metadata
df = pd.read_csv(ANN_PATH)
print(df['dx'].value_counts())


dx
melanocytic nevi                 6705
melanoma                         1112
benign keratosis-like lesions    1098
basal cell carcinoma              514
actinic keratoses                 327
vascular lesions                  142
dermatofibroma                    115
Name: count, dtype: int64


In [10]:
# Map labels to numbers
label_map = {label: i for i, label in enumerate(df['dx'].unique())}
df['label'] = df['dx'].map(label_map)

In [11]:
# Train-test split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)


In [12]:
# Augmentations
train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(),
    A.VerticalFlip(),
    A.RandomBrightnessContrast(),
    A.Normalize(),
    ToTensorV2(),
])

test_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(),
    ToTensorV2(),
])

class SkinDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['image_id'] + ".jpg")
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = row['label']
        if self.transform:
            image = self.transform(image=image)['image']
        return image, label

train_dataset = SkinDataset(train_df, IMG_DIR, transform=train_transform)
test_dataset = SkinDataset(test_df, IMG_DIR, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [13]:
import torch.nn as nn
import torchvision.models as models

num_classes = len(label_map)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)  # replace final layer
model = model.to(device)


/home/the/miniconda/envs/ml2/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/the/miniconda/envs/ml2/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/the/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:08<00:00, 11.7MB/s]


In [14]:
import torch.optim as optim
from tqdm import tqdm

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for imgs, labels in tqdm(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss/len(train_loader):.4f}")


100%|██████████| 251/251 [01:04<00:00,  3.91it/s]


Epoch 1/10, Loss: 0.6616


100%|██████████| 251/251 [00:59<00:00,  4.24it/s]


Epoch 2/10, Loss: 0.4670


100%|██████████| 251/251 [00:59<00:00,  4.23it/s]


...

Epoch 8/10, Loss: 0.1980


100%|██████████| 251/251 [00:59<00:00,  4.25it/s]


Epoch 9/10, Loss: 0.1617


100%|██████████| 251/251 [00:59<00:00,  4.24it/s]

Epoch 10/10, Loss: 0.1411


In [15]:
from sklearn.metrics import classification_report

model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        preds = outputs.argmax(dim=1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print(classification_report(y_true, y_pred, target_names=list(label_map.keys())))



                               precision    recall  f1-score   support

             melanocytic nevi       0.94      0.90      0.92      1341
                     melanoma       0.54      0.78      0.64       223
benign keratosis-like lesions       0.78      0.61      0.68       220
         basal cell carcinoma       0.89      0.54      0.67       103
            actinic keratoses       0.49      0.75      0.60        65
             vascular lesions       0.87      0.96      0.92        28
               dermatofibroma       0.79      0.65      0.71        23

                     accuracy                           0.83      2003
                    macro avg       0.76      0.74      0.73      2003
                 weighted avg       0.85      0.83      0.84      2003



In [16]:
from torchcam.methods import GradCAM
from torchcam.utils import overlay_mask
from torchvision.transforms.functional import to_pil_image
import matplotlib.pyplot as plt

cam_extractor = GradCAM(model, target_layer='layer4')

model.eval()
img, label = test_dataset[0]
img = img.unsqueeze(0).to(device)

out = model(img)
pred = out.argmax(dim=1).item()

activation_map = cam_extractor(pred, out)
result = overlay_mask(to_pil_image(img.squeeze()), to_pil_image(activation_map[0].squeeze(), mode='F'), alpha=0.5)
plt.imshow(result)
plt.title(f"Predicted: {list(label_map.keys())[pred]}")
plt.show()



ModuleNotFoundError: No module named 'torchcam'

In [17]:
import torch
import json

# Save model weights
torch.save(model.state_dict(), "melanoma_resnet50.pth")

# Save checkpoint (if you want to resume training later)
torch.save({
    "epoch": EPOCHS,
    "model_state": model.state_dict(),
    "optimizer_state": optimizer.state_dict(),
    "loss": total_loss,
}, "checkpoint.pth")

# Save label map
import json
with open("label_map.json", "w") as f:
    json.dump(label_map, f)
